In [9]:
# -*- coding: utf-8 -*-
import cv2
import mediapipe as mp
import numpy as np
import requests
import math
import time
import threading
from queue import Queue, Empty
from scipy.ndimage import gaussian_filter1d
import uuid
from typing import List, Tuple, Optional
import paho.mqtt.client as mqtt

# Khởi tạo MediaPipe với model nhẹ hơn
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
# *** OPTIMIZATION: Use model_complexity=0 for potentially faster processing ***
hands = mp_hands.Hands(model_complexity=0, max_num_hands=1, min_detection_confidence=0.7, min_tracking_confidence=0.5)

# Địa chỉ server
SERVER_URL = "http://localhost:8000/control"

# --- MQTT Configuration ---
MQTT_BROKER = "localhost"  # Địa chỉ IP hoặc hostname của MQTT broker (ví dụ: Mosquitto)
MQTT_PORT = 1883           # Port MQTT mặc định
MQTT_TOPIC = "mpu6050/alert" # Topic để publish lệnh điều khiển
MQTT_CLIENT_ID = f"gesture_controller_{uuid.uuid4()}" # Tạo ID client duy nhất

# Hằng số (Xem xét lại các giá trị này nếu cần)
MAX_POINTS = 65
MIN_POINTS_CIRCLE = 10
MIN_POINTS_RECTANGLE = 12
MIN_POINTS_TRIANGLE = 15
COMMAND_COOLDOWN = 0.5 # Thời gian chờ giữa các lệnh
MIN_RADIUS = 20
MIN_SIDE_LENGTH = 20
MIN_AREA = 500
STARTUP_DURATION = 1.5 # Giảm thời gian khởi động nếu muốn
CLOSURE_THRESHOLD = 30 # Tăng nhẹ nếu khó khép kín
MIN_POINT_DISTANCE = 5 # Khoảng cách tối thiểu giữa các điểm liên tiếp
HAND_DETECTION_DRAW_DELAY = 0.75 # <<<--- NEW: Thời gian chờ sau khi phát hiện tay
NO_HAND_TIMEOUT = 1.0 # Thời gian không thấy tay trước khi reset điểm vẽ

command_queue = Queue(maxsize=10)
mqtt_connected = False 
mqtt_client = None  # Added global variable for the MQTT client

# Biến lưu trữ điểm ngón tay
points: List[Tuple[int, int]] = []

# --- MQTT Callback Functions ---
def on_connect(client, userdata, flags, rc):
    """Callback khi kết nối MQTT thành công."""
    global mqtt_connected
    if rc == 0:
        print(f"Connected to MQTT Broker: {MQTT_BROKER}")
        mqtt_connected = True
    else:
        print(f"Failed to connect to MQTT Broker, return code {rc}")
        mqtt_connected = False

def on_disconnect(client, userdata, rc):
    """Callback khi mất kết nối MQTT."""
    global mqtt_connected
    print(f"Disconnected from MQTT Broker (rc: {rc}). Will attempt to reconnect.")
    mqtt_connected = False
    # Paho client sẽ tự động thử kết nối lại nếu loop đang chạy

def on_publish(client, userdata, mid):
    """Callback sau khi publish (ít dùng hơn)."""
    # print(f"Message Published (mid={mid})")
    pass

# --- MQTT setup function ---
def setup_mqtt():
    """Thiết lập và kết nối MQTT client."""
    global mqtt_client
    
    try:
        client = mqtt.Client(client_id=MQTT_CLIENT_ID)
        client.username_pw_set(username="hiep", password="1234") 
        client.on_connect = on_connect
        client.on_disconnect = on_disconnect
        client.on_publish = on_publish
        
        # Connect to the broker
        client.connect(MQTT_BROKER, MQTT_PORT, keepalive=60)
        
        # Start the loop in a non-blocking way
        client.loop_start()
        
        mqtt_client = client
        return client
    except Exception as e:
        print(f"Error setting up MQTT: {e}")
        return None

# --- MQTT publish function ---
def publish_mqtt_alert(command):
    """Gửi thông báo qua MQTT."""
    global mqtt_client, mqtt_connected
    
    if not mqtt_client or not mqtt_connected:
        print("MQTT client not connected, can't publish alert")
        return False
    
    try:
        msg_info = mqtt_client.publish(MQTT_TOPIC, command, qos=1)
        msg_info.wait_for_publish(timeout=2.0)
        print(f"MQTT Alert Published: {command}")
        return True
    except Exception as e:
        print(f"Error publishing MQTT alert: {e}")
        return False

# --- Các hàm tiện ích (giữ nguyên) ---
def smooth_points(pts: List[Tuple[int, int]], sigma: float = 1.5) -> List[Tuple[int, int]]:
    """Hàm làm mượt đường vẽ."""
    try:
        if len(pts) < 2:
            return pts
        np_pts = np.array(pts, dtype=np.float32)
        smoothed_x = gaussian_filter1d(np_pts[:, 0], sigma=sigma)
        smoothed_y = gaussian_filter1d(np_pts[:, 1], sigma=sigma)
        return [(int(x), int(y)) for x, y in zip(smoothed_x, smoothed_y)]
    except Exception as e:
        print(f"Error in smooth_points: {e}")
        return pts

def calculate_angle(p1: Tuple[int, int], p2: Tuple[int, int], p3: Tuple[int, int]) -> float:
    """Hàm tính góc giữa ba điểm."""
    try:
        v1 = np.array([p1[0] - p2[0], p1[1] - p2[1]])
        v2 = np.array([p3[0] - p2[0], p3[1] - p2[1]])
        norm_v1 = np.linalg.norm(v1)
        norm_v2 = np.linalg.norm(v2)
        if norm_v1 < 1e-6 or norm_v2 < 1e-6:
            return 0.0
        dot_product = np.dot(v1, v2)
        cos_theta = np.clip(dot_product / (norm_v1 * norm_v2), -1.0, 1.0)
        angle = math.degrees(math.acos(cos_theta))
        return angle
    except Exception as e:
        print(f"Error in calculate_angle: {e}")
        return 0.0

def calculate_area(pts: List[Tuple[int, int]]) -> float:
    """Hàm tính diện tích đa giác (Shoelace formula)."""
    try:
        if len(pts) < 3:
            return 0.0
        area = 0.0
        n = len(pts)
        for i in range(n):
            j = (i + 1) % n
            area += pts[i][0] * pts[j][1]
            area -= pts[j][0] * pts[i][1]
        return abs(area) / 2.0
    except Exception as e:
        print(f"Error in calculate_area: {e}")
        return 0.0

def is_closed_shape(pts: List[Tuple[int, int]], threshold: float = CLOSURE_THRESHOLD) -> bool:
    """Hàm kiểm tra tính khép kín."""
    try:
        if len(pts) < 3: # Cần ít nhất 3 điểm để có thể coi là khép kín
            return False
        first_point = pts[0]
        last_point = pts[-1]
        distance = math.dist(first_point, last_point)
        return distance < threshold
    except Exception as e:
        print(f"Error in is_closed_shape: {e}")
        return False

def is_circle(pts: List[Tuple[int, int]], tolerance: float = 0.25) -> bool:
    """Hàm kiểm tra hình tròn."""
    try:
        if len(pts) < MIN_POINTS_CIRCLE or not is_closed_shape(pts):
            return False
        np_pts = np.array(pts, dtype=np.float32)
        center, radius = cv2.minEnclosingCircle(np_pts)
        if radius < MIN_RADIUS:
            return False
        area = calculate_area(pts)
        if area < MIN_AREA:
             return False
        theoretical_area = math.pi * radius * radius
        if theoretical_area < 1e-6: # Avoid division by zero
             return False
        area_ratio = area / theoretical_area
        if not (0.6 < area_ratio < 1.4):
             return False
        distances = [math.dist((x, y), center) for (x, y) in pts]
        avg_distance = np.mean(distances)
        if avg_distance < 1e-6: # Avoid division by zero
            return False
        max_deviation = np.max(np.abs(distances - avg_distance))
        return (max_deviation / avg_distance) < tolerance # Check deviation relative to average distance
    except cv2.error as e:
        print(f"OpenCV Error in is_circle (likely not enough points for minEnclosingCircle): {e}")
        return False
    except Exception as e:
        print(f"Error in is_circle: {e}")
        return False

def is_rectangle(pts: List[Tuple[int, int]], angle_tolerance: float = 20) -> bool:
    """Hàm kiểm tra hình chữ nhật."""
    try:
        if len(pts) < MIN_POINTS_RECTANGLE or not is_closed_shape(pts):
            return False
        np_pts = np.array(pts, dtype=np.float32)
        perimeter = cv2.arcLength(np_pts, True)
        if perimeter < 4 * MIN_SIDE_LENGTH:
            return False

        epsilon = 0.035 * perimeter # Tăng nhẹ epsilon
        approx = cv2.approxPolyDP(np_pts, epsilon, True)

        if len(approx) != 4:
            return False

        approx_pts = [tuple(p[0]) for p in approx]
        area = calculate_area(approx_pts)
        if area < MIN_AREA:
            return False

        angles = []
        sides = []
        for i in range(4):
            p1 = approx_pts[i]
            p2 = approx_pts[(i + 1) % 4]
            p3 = approx_pts[(i + 2) % 4]
            angle = calculate_angle(p1, p2, p3)
            angles.append(angle)
            side_length = math.dist(p1, p2)
            sides.append(side_length)

        if min(sides) < MIN_SIDE_LENGTH:
            return False
        angle_deviation = np.max(np.abs(np.array(angles) - 90))
        if angle_deviation > angle_tolerance:
             return False
        return True # Passed all checks
    except Exception as e:
        print(f"Error in is_rectangle: {e}")
        return False

def is_triangle(pts: List[Tuple[int, int]]) -> bool:
    """Hàm kiểm tra tam giác."""
    try:
        if len(pts) < MIN_POINTS_TRIANGLE or not is_closed_shape(pts):
            return False
        np_pts = np.array(pts, dtype=np.float32)
        perimeter = cv2.arcLength(np_pts, True)
        if perimeter < 3 * MIN_SIDE_LENGTH:
             return False

        epsilon = 0.05 * perimeter # Tăng nhẹ epsilon
        approx = cv2.approxPolyDP(np_pts, epsilon, True)

        if len(approx) != 3:
            return False

        approx_pts = [tuple(p[0]) for p in approx]
        area = calculate_area(approx_pts)
        if area < MIN_AREA:
            return False

        angles = []
        sides = []
        for i in range(3):
            p1 = approx_pts[i]
            p2 = approx_pts[(i + 1) % 3]
            p3 = approx_pts[(i + 2) % 3]
            angle = calculate_angle(p1, p2, p3)
            angles.append(angle)
            side_length = math.dist(p1, p2)
            sides.append(side_length)

        if min(sides) < MIN_SIDE_LENGTH:
            return False
        if min(angles) < 20 or max(angles) > 160:
             return False
        return True # Passed all checks
    except Exception as e:
        print(f"Error in is_triangle: {e}")
        return False

# --- Gửi lệnh và Xử lý Queue ---
def send_command(command: str) -> bool:
    """Gửi lệnh qua HTTP."""
    try:
        print(f"Attempting to send: {command}")
        payload = {"cmd": command, "id": str(uuid.uuid4())}
        response = requests.post(SERVER_URL, json=payload, timeout=1.0)
        print(f"Sent: {command} -> Server Response: {response.status_code}")
        
        # Thêm gửi thông qua MQTT
        if command in ["CIRCLE", "RECTANGLE"]:
            publish_mqtt_alert(command)
            
        return response.ok
    except requests.exceptions.RequestException as e:
        print(f"Network error sending command: {e}")
        return False
    except Exception as e:
        print(f"Error in send_command: {e}")
        return False

def clear_queue(q: Queue):
    """Hàm xóa hàng đợi an toàn."""
    while True:
        try:
            q.get_nowait()
            q.task_done()
        except Empty:
            break

def command_processor(q: Queue):
    """Thread xử lý lệnh từ queue."""
    last_command_time = 0
    while True:
        try:
            command = q.get()
            if command == "EXIT":
                print("Command processor received EXIT signal.")
                break

            current_time = time.time()
            if current_time - last_command_time >= COMMAND_COOLDOWN:
                if send_command(command):
                    last_command_time = current_time
                else:
                    print(f"Failed to send command '{command}', maybe retry later?")
            else:
                print(f"Command '{command}' skipped due to cooldown.")
            q.task_done()
        except Exception as e:
            print(f"Error in command_processor: {e}")
            try:
                q.task_done()
            except ValueError:
                pass


# Hàm thêm điểm với kiểm tra khoảng cách
def add_point(cx: int, cy: int, min_distance: float = MIN_POINT_DISTANCE):
    """Thêm điểm vào danh sách nếu đủ xa điểm cuối."""
    global points
    if not points or math.dist(points[-1], (cx, cy)) > min_distance:
        points.append((cx, cy))
        if len(points) > MAX_POINTS:
            points.pop(0)

# --- Main Loop ---
def main():
    global points # Khai báo để hàm add_point có thể sửa đổi

    # Thiết lập MQTT
    setup_mqtt()

    # Khởi động thread xử lý lệnh
    cmd_thread = threading.Thread(target=command_processor, args=(command_queue,), daemon=True)
    cmd_thread.start()

    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Cannot open camera")
        return

    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640*2)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480*2)

    shape_sent_this_gesture = False
    is_drawing_locked = False
    prev_frame_time = 0.0
    startup_time = time.time()
    is_startup = True
    status_text = "Initializing..."
    last_hand_detected_time = 0 # Vẫn dùng để reset points khi tay mất
    hand_detected_start_time: Optional[float] = None # <<<--- NEW: Thời điểm bắt đầu phát hiện tay liên tục
    can_start_drawing = False # <<<--- NEW: Cờ cho phép bắt đầu vẽ sau delay

    while True:
        try:
            ret, frame = cap.read()
            if not ret:
                print("Error: Cannot read frame")
                time.sleep(0.1)
                continue

            current_frame_time = time.time()
            fps = 1.0 / (current_frame_time - prev_frame_time) if prev_frame_time > 0 else 0
            prev_frame_time = current_frame_time

            frame = cv2.flip(frame, 1)
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            rgb_frame.flags.writeable = False
            results = hands.process(rgb_frame)
            rgb_frame.flags.writeable = True

            # Hiển thị thông tin cơ bản (cập nhật status ở logic dưới)
            cv2.rectangle(frame, (5, 5), (450, 90), (0, 0, 0), -1)
            cv2.putText(frame, f"FPS: {int(fps)}", (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            cv2.putText(frame, f"Points: {len(points)}", (150, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
            
            # Hiển thị trạng thái MQTT
            mqtt_status = "MQTT: Connected" if mqtt_connected else "MQTT: Disconnected"
            cv2.putText(frame, mqtt_status, (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255) if mqtt_connected else (0, 0, 255), 2)

            # --- Giai đoạn khởi động ---
            if is_startup:
                if current_frame_time - startup_time > STARTUP_DURATION:
                    is_startup = False
                    status_text = "Show hand to draw shape"
                else:
                    status_text = f"Starting... {STARTUP_DURATION - (current_frame_time - startup_time):.1f}s"
                # Vẫn xử lý hiển thị tay nếu có trong lúc khởi động
                if results.multi_hand_landmarks:
                     for hand_landmarks in results.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                cv2.putText(frame, status_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
                cv2.imshow("Draw with finger", frame)
                if cv2.waitKey(1) & 0xFF == ord('q'):
                     break
                continue # Bỏ qua phần còn lại khi đang khởi động

            # --- Xử lý chính: Vẽ và Nhận diện ---
            hand_currently_detected = bool(results.multi_hand_landmarks)

            if hand_currently_detected:
                last_hand_detected_time = current_frame_time # Cập nhật thời gian thấy tay (cho logic reset)
                hand_landmarks = results.multi_hand_landmarks[0]

                # Vẽ khung xương tay
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

                # --- Logic chờ trước khi vẽ ---
                if hand_detected_start_time is None:
                    # Đây là lần đầu tiên phát hiện tay trong chuỗi này
                    hand_detected_start_time = current_frame_time
                    can_start_drawing = False # Chưa cho vẽ ngay
                    points.clear() # Xóa điểm cũ khi tay mới xuất hiện
                    shape_sent_this_gesture = False # Reset cờ gửi
                    is_drawing_locked = False # Mở khóa vẽ (nếu đang khóa)
                    status_text = f"Hand detected. Waiting {HAND_DETECTION_DRAW_DELAY:.2f}s..."
                    print("Hand detected, starting wait timer.")

                # Kiểm tra xem đã hết thời gian chờ chưa
                wait_elapsed = current_frame_time - hand_detected_start_time
                if not can_start_drawing and wait_elapsed >= HAND_DETECTION_DRAW_DELAY:
                    can_start_drawing = True
                    status_text = "Ready to draw!"
                    print("Wait time elapsed. Drawing enabled.")

                # --- Chỉ xử lý vẽ và nhận diện nếu ĐÃ HẾT CHỜ và CHƯA BỊ KHÓA ---
                if can_start_drawing and not is_drawing_locked:
                    status_text = "Drawing..." # Cập nhật trạng thái
                    # Lấy tọa độ đầu ngón trỏ
                    index_finger_tip = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
                    h, w, _ = frame.shape
                    cx, cy = int(index_finger_tip.x * w), int(index_finger_tip.y * h)

                    # Thêm điểm mới
                    add_point(cx, cy)

                    # Làm mượt và vẽ đường vẽ
                    smoothed_points = smooth_points(points)
                    if len(smoothed_points) > 1:
                        cv2.polylines(frame, [np.array(smoothed_points)], isClosed=False, color=(255, 0, 255), thickness=3)

                    # --- Nhận diện hình dạng ---
                    min_req_points = min(MIN_POINTS_CIRCLE, MIN_POINTS_RECTANGLE, MIN_POINTS_TRIANGLE)
                    if len(smoothed_points) >= min_req_points and not shape_sent_this_gesture:
                        command: Optional[str] = None
                        detected_shape = ""

                        if is_closed_shape(smoothed_points):
                            if is_circle(smoothed_points):
                                command = "CIRCLE"
                                detected_shape = "Circle"
                            elif is_rectangle(smoothed_points):
                                command = "RECTANGLE"
                                detected_shape = "Rectangle"
                            elif is_triangle(smoothed_points):
                                command = "TRIANGLE"
                                detected_shape = "Triangle"

                        if command:
                            try:
                                command_queue.put_nowait(command)
                                status_text = f"{detected_shape} Detected! Press 'r' to reset."
                                print(f"Shape '{command}' detected, adding to queue.")
                                
                                # Directly publish to MQTT for immediate feedback without waiting for command_processor
                                if command in ["CIRCLE", "RECTANGLE"]:
                                    publish_mqtt_alert(command)
                                    
                                shape_sent_this_gesture = True
                                is_drawing_locked = True # Khóa vẽ/nhận diện lại
                                points.clear() # Xóa điểm sau khi nhận dạng
                                # can_start_drawing = False # Không cần reset cái này ở đây, chỉ reset khi mất tay hoặc nhấn 'r'
                            except Exception as e:
                                status_text = "Command queue full!"
                                print(f"Could not add command to queue: {e}")
                elif is_drawing_locked:
                    # Đã khóa vẽ, chờ reset
                    status_text = f"{command} sent. Press 'r' to draw again."
                elif not can_start_drawing:
                    # Vẫn đang trong giai đoạn chờ
                    remaining_wait = HAND_DETECTION_DRAW_DELAY - wait_elapsed
                    status_text = f"Waiting... {remaining_wait:.1f}s"

            else:
                # --- Không phát hiện tay ---
                if hand_detected_start_time is not None:
                    # Tay vừa bị mất
                    print("Hand lost.")
                    hand_detected_start_time = None # Reset thời điểm bắt đầu
                    can_start_drawing = False # Ngừng cho phép vẽ

                # Logic xóa điểm sau một khoảng thời gian không thấy tay (giữ nguyên)
                if current_frame_time - last_hand_detected_time > NO_HAND_TIMEOUT and len(points) > 0:
                    print("No hand detected for a while, clearing points.")
                    points.clear()
                    # shape_sent_this_gesture = False # Đã reset ở trên khi hand_detected_start_time = None

                if not is_drawing_locked:
                     status_text = "No hand detected. Show hand."
                # else: Giữ nguyên status "Shape sent. Press 'r'..." nếu đang khóa

            # Cập nhật và hiển thị status_text cuối cùng
            cv2.putText(frame, status_text, (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.imshow("Draw with finger", frame)

            # --- Xử lý phím bấm ---
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("'q' pressed, exiting...")
                try:
                    command_queue.put_nowait("EXIT")
                except Exception as e:
                    print(f"Could not send EXIT signal to queue: {e}")
                break
            elif key == ord('r'):
                print("'r' pressed, resetting...")
                points.clear()
                clear_queue(command_queue)
                shape_sent_this_gesture = False
                is_drawing_locked = False
                can_start_drawing = False # Reset trạng thái cho phép vẽ
                hand_detected_start_time = None # Reset timer chờ
                status_text = "Ready to draw. Show hand."
            # NEW: Phím tắt để kiểm tra MQTT
            elif key == ord('m'):
                print("'m' pressed, testing MQTT...")
                test_result = publish_mqtt_alert("TEST")
                status_text = f"MQTT Test: {'Success' if test_result else 'Failed'}"

        except Exception as e:
            print(f"!!!!!!!!!! UNEXPECTED ERROR in main loop: {e} !!!!!!!!!!!")
            import traceback
            traceback.print_exc()
            status_text = "Error occurred. Trying to continue..."
            # Reset trạng thái để thử lại an toàn hơn
            points.clear()
            shape_sent_this_gesture = False
            is_drawing_locked = False
            can_start_drawing = False
            hand_detected_start_time = None
            time.sleep(0.5)

    # --- Dọn dẹp ---
    print("Releasing camera...")
    cap.release()
    print("Destroying windows...")
    cv2.destroyAllWindows()
    
    # Cleanup MQTT connection
    if mqtt_client:
        print("Disconnecting MQTT client...")
        mqtt_client.loop_stop()
        mqtt_client.disconnect()
        
    print("Waiting for command processor thread to finish...")
    cmd_thread.join(timeout=2.0)
    print("Exited gracefully.")

if __name__ == "__main__":
    main()

C:\Users\DELL\AppData\Local\Temp\ipykernel_6476\827792431.py:81: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  client = mqtt.Client(client_id=MQTT_CLIENT_ID)


Connected to MQTT Broker: localhost
Hand detected, starting wait timer.
Wait time elapsed. Drawing enabled.
Shape 'RECTANGLE' detected, adding to queue.Attempting to send: RECTANGLE

MQTT Alert Published: RECTANGLE
Network error sending command: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /control (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001C5F66515D0>, 'Connection to localhost timed out. (connect timeout=1.0)'))
Failed to send command 'RECTANGLE', maybe retry later?
'r' pressed, resetting...
Hand detected, starting wait timer.
Wait time elapsed. Drawing enabled.
Shape 'CIRCLE' detected, adding to queue.Attempting to send: CIRCLE

MQTT Alert Published: CIRCLE
Network error sending command: HTTPConnectionPool(host='localhost', port=8000): Max retries exceeded with url: /control (Caused by ConnectTimeoutError(<urllib3.connection.HTTPConnection object at 0x000001C5D2C4B1D0>, 'Connection to localhost timed out. (co